## Building a service server

This is an exact replication of creating an service ([here](https://docs.ros.org/en/galactic/Tutorials/Beginner-Client-Libraries/Writing-A-Simple-Py-Service-And-Client.html))  The first step is to define it by creating the requisite paths, generate all the folders, and create a package.

In [1]:
import os
import subprocess

home = os.path.expanduser('~')
workspace = home +'/ros2_ws/'
sourceFiles = workspace + 'src/'
packageName = 'py_srvcli'

os.chdir(home)

if not os.path.exists(sourceFiles):
    os.makedirs(sourceFiles)
os.chdir(sourceFiles)
if not os.path.exists(packageName):
    reply = subprocess.run(['ros2','pkg','create','--build-type','ament_python',packageName,'--dependencies','rclpy','example_interfaces'],capture_output=True) #same as !ros2 pkg create --build-type ament_python py_srvcli --dependencies rclpy example_interfaces

In [ ]:
reply

Next we set up the service definition file.  This defines the data types for *Requests* and  *Response*, separated by a dashed line.  In this case, the request has two parameters, a and b, both 64 bit integers, the result (only one parameter) is also a 64 bit integers.  First change the directory to the service folder and then save it.

In [2]:
os.chdir(sourceFiles + packageName)

In [3]:
%%writefile add.srv

int64 a
int64 b
---
int64 sum

Writing add.srv


We then should update the package xml, adding your name and such.
```xml
<description>Python client server tutorial</description>
<maintainer email="you@email.com">Your Name</maintainer>
<license>Apache License 2.0</license>
```

In [4]:
os.chdir(sourceFiles + packageName)

In [5]:
%%writefile package.xml
<?xml version="1.0"?>
<?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
<package format="3">
  <name>py_srvcli</name>
  <version>0.0.0</version>
  <description>TODO: Package description</description>
  <maintainer email="parallels@todo.todo">parallels</maintainer>
  <license>TODO: License declaration</license>

  <depend>rclpy</depend>
  <depend>example_interfaces</depend>

  <test_depend>ament_copyright</test_depend>
  <test_depend>ament_flake8</test_depend>
  <test_depend>ament_pep257</test_depend>
  <test_depend>python3-pytest</test_depend>

  <export>
    <build_type>ament_python</build_type>
  </export>
</package>

Overwriting package.xml


Likewise, you shoudl update the setup.py file
```python
maintainer='Your Name',
maintainer_email='you@email.com',
description='Python client server tutorial',
license='Apache License 2.0',
```

In [6]:
os.chdir(sourceFiles + packageName)

In [7]:
%%writefile setup.py
from setuptools import setup

package_name = 'py_srvcli'

setup(
    name=package_name,
    version='0.0.0',
    packages=[package_name],
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='parallels',
    maintainer_email='parallels@todo.todo',
    description='TODO: Package description',
    license='TODO: License declaration',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
        ],
    },
)

Overwriting setup.py


In [14]:
os.chdir(sourceFiles + packageName + '/' + packageName)

In [15]:
%%writefile service_member_function.py
from example_interfaces.srv import AddTwoInts

import rclpy
from rclpy.node import Node


class MinimalService(Node):

    def __init__(self):
        super().__init__('minimal_service')
        self.srv = self.create_service(AddTwoInts, 'add_two_ints', self.add_two_ints_callback)

    def add_two_ints_callback(self, request, response):
        response.sum = request.a + request.b
        self.get_logger().info('Incoming request\na: %d b: %d' % (request.a, request.b))

        return response


def main():
    rclpy.init()

    minimal_service = MinimalService()

    rclpy.spin(minimal_service)

    rclpy.shutdown()


if __name__ == '__main__':
    main()

Writing service_member_function.py


In [17]:
%%writefile client_member_function.py
import sys

from example_interfaces.srv import AddTwoInts
import rclpy
from rclpy.node import Node


class MinimalClientAsync(Node):

    def __init__(self):
        super().__init__('minimal_client_async')
        self.cli = self.create_client(AddTwoInts, 'add_two_ints')
        while not self.cli.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('service not available, waiting again...')
        self.req = AddTwoInts.Request()

    def send_request(self, a, b):
        self.req.a = a
        self.req.b = b
        self.future = self.cli.call_async(self.req)
        rclpy.spin_until_future_complete(self, self.future)
        return self.future.result()


def main():
    rclpy.init()

    minimal_client = MinimalClientAsync()
    response = minimal_client.send_request(int(sys.argv[1]), int(sys.argv[2]))
    minimal_client.get_logger().info(
        'Result of add_two_ints: for %d + %d = %d' %
        (int(sys.argv[1]), int(sys.argv[2]), response.sum))

    minimal_client.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()

Writing client_member_function.py


In [18]:
os.chdir(sourceFiles + packageName)

In [19]:
%%writefile setup.py
from setuptools import setup

package_name = 'py_srvcli'

setup(
    name=package_name,
    version='0.0.0',
    packages=[package_name],
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='parallels',
    maintainer_email='parallels@todo.todo',
    description='TODO: Package description',
    license='TODO: License declaration',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
            'service = py_srvcli.service_member_function:main',
            'client = py_srvcli.client_member_function:main',
        ],
    },
)

Overwriting setup.py


In [20]:
os.chdir(workspace)
!rosdep install -i --from-path src --rosdistro galactic -y

#All required rosdeps installed successfully


Lastly we have to build it by running *colcon build* in the workspace directory

In [21]:
import subprocess
os.chdir(workspace)
subprocess.run(['colcon','build','--packages-select','py_srvcli'],capture_output=True)

Now set everything up to run in a terminal
```bash
cd ~/ros2_ws
. install/setup.bash
export ROS_DOMAIN_ID=0
ros2 run py_srvcli service
```
in one terminal and 
```bash
cd ~/ros2_ws
. install/setup.bash
export ROS_DOMAIN_ID=0
ros2 run py_srvcli client 2 3
```
in the other